In [1]:
import pandas as pd
import numpy as np

In [2]:
# Reads in dataset
data = pd.read_csv('NY property data.csv')
data.head()

,RECORD,BBLE,B,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,...,BLDFRONT,BLDDEPTH,AVLAND2,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE
0,1,1000010101,1,1,101,NaN,U S GOVT LAND & BLDGS,P7,4,500,...,0,0,3775500.0,8613000.0,3775500.0,8613000.0,NaN,FINAL,2010/11,AC-TR
1,2,1000010201,1,1,201,NaN,U S GOVT LAND & BLDGS,Z9,4,27,...,0,0,11111400.0,80690400.0,11111400.0,80690400.0,NaN,FINAL,2010/11,AC-TR
2,3,1000020001,1,2,1,NaN,DEPT OF GENERAL SERVI,Y7,4,709,...,709,564,32321790.0,40179510.0,32321790.0,40179510.0,NaN,FINAL,2010/11,AC-TR
3,4,1000020023,1,2,23,NaN,DEPARTMENT OF BUSINES,T2,4,793,...,85,551,13644000.0,15750000.0,13644000.0,15750000.0,NaN,FINAL,2010/11,AC-TR
4,5,1000030001,1,3,1,NaN,PARKS AND RECREATION,Q1,4,323,...,89,57,106348680.0,107758350.0,106348680.0,107758350.0,NaN,FINAL,2010/11,AC-TR


In [3]:
data.dtypes

RECORD        int64
BBLE         object
B             int64
BLOCK         int64
LOT           int64
EASEMENT     object
OWNER        object
BLDGCL       object
TAXCLASS     object
LTFRONT       int64
LTDEPTH       int64
EXT          object
STORIES     float64
FULLVAL     float64
AVLAND      float64
AVTOT       float64
EXLAND      float64
EXTOT       float64
EXCD1       float64
STADDR       object
ZIP         float64
EXMPTCL      object
BLDFRONT      int64
BLDDEPTH      int64
AVLAND2     float64
AVTOT2      float64
EXLAND2     float64
EXTOT2      float64
EXCD2       float64
PERIOD       object
YEAR         object
VALTYPE      object
dtype: object

# Filling in Missing Values

### Filling in Missing Zip Codes

In [4]:
# Counts number of values where zip code is missing
missing_zips = np.where(pd.isnull(data['ZIP']))[0]
len(missing_zips)

29890

In [5]:
# Loops through data set and adds fills missing zip codes that are inbetween two observations with matching zip codes
for i in range(len(missing_zips)):
    if(data.loc[missing_zips[i]+1, 'ZIP'] == data.loc[missing_zips[i]-1, 'ZIP']):
        data.loc[missing_zips[i], 'ZIP'] = data.loc[missing_zips[i]-1, 'ZIP']

In [6]:
# Counts the remaining amount of observations that are missing zip codes
missing_zips = np.where(pd.isnull(data['ZIP']))[0]
len(missing_zips)

16437

In [7]:
# Loops through and fills missing zip codes with prior observation zip code
for i in range(len(missing_zips)):
    data.loc[missing_zips[i], 'ZIP'] = data.loc[missing_zips[i]-1,'ZIP']

In [8]:
# Counts how many remaining missing zip codes there are
missing_zips = np.where(pd.isnull(data['ZIP']))[0]
len(missing_zips)

0

### Filling in MIssing STORIES

In [9]:
# Counts the number of observations that are missing stories
temp = data[data['STORIES'].isnull()]
len(temp)

56264

In [10]:
# counts the amount of observations that belong to a specific class
temp['TAXCLASS'].value_counts()

1B    24736
4     22354
3      4635
2      3435
1       897
2C      138
2B       34
2A       30
1A        5
Name: TAXCLASS, dtype: int64

In [11]:
# finds the mean stories of each TAXCLASS
mean_stories = data.groupby('TAXCLASS')['STORIES'].mean()
print(mean_stories)

TAXCLASS
1      2.111641
1A     1.656837
1B     4.000000
1C     3.052748
1D     1.068966
2     16.095110
2A     2.844574
2B     4.004494
2C     4.745097
3      1.333333
4      5.446968
Name: STORIES, dtype: float64


In [12]:
# Checks how many observations have zero stories
len(data[data['STORIES'] == 0])

0

In [13]:
# Fills NA values with zero for Stories
data['STORIES'] = data['STORIES'].fillna(value=0)

# Loops through dataset and fills stories based on the average from the TAX CLASS
for index in mean_stories.index:
    data.loc[(data['STORIES']==0)&(data['TAXCLASS']==index), 'STORIES'] = mean_stories[index]

### Filling in Missing FULLVAL, AVLAND, AVTOT

In [14]:
# Replaces NA values with a zero 
data['FULLVAL'].replace('NaN', 0)

# Finds the average FULLVAL base on taxclass
temp = data[data['FULLVAL']!=0]
mean_fullval = temp.groupby('TAXCLASS')['FULLVAL'].mean()
print(mean_fullval)

TAXCLASS
1     5.698435e+05
1A    3.352842e+05
1B    5.613639e+05
1C    7.615359e+05
1D    2.233614e+07
2     7.998018e+05
2A    8.640037e+05
2B    1.252989e+06
2C    7.728799e+05
3     1.112765e+05
4     3.211928e+06
Name: FULLVAL, dtype: float64


In [15]:
# Replaces NA values with a zero
data['AVLAND'].replace('NaN', 0)

# Finds the average FULLVAL base on taxclass
temp_avland = data[data['AVLAND']!=0]
mean_avland = temp_avland.groupby('TAXCLASS')['AVLAND'].mean()

In [16]:
# Replaces NA values with a zero
data['AVTOT'].replace('NaN', 0)

# Finds the average FULLVAL base on taxclass
temp_avtot = data[data['AVTOT']!=0]
mean_avtot = temp_avtot.groupby('TAXCLASS')['AVTOT'].mean()

In [17]:
%%time
# Loops through dataset and inserts the mean value based on the average tax class
for index in mean_fullval.index:
    data.loc[(data['FULLVAL'] == 0)&(data["TAXCLASS"]==index), 'FULLVAL'] = mean_fullval[index]
    data.loc[(data['AVLAND'] == 0)&(data["TAXCLASS"]==index), 'AVLAND'] = mean_avland[index]
    data.loc[(data['AVTOT'] == 0)&(data["TAXCLASS"]==index), 'AVTOT'] = mean_avtot[index]

CPU times: user 1.64 s, sys: 48 ms, total: 1.69 s
Wall time: 1.69 s


### Fill in Missing Lot and Building Sizes

In [18]:
# Sets missing values to NAN 
data.loc[data['LTFRONT']==0,'LTFRONT']=np.nan
data.loc[data['LTDEPTH']==0,'LTDEPTH']=np.nan
data.loc[data['BLDFRONT']==0,'BLDFRONT']=np.nan
data.loc[data['BLDDEPTH']==0,'BLDDEPTH']=np.nan
data.loc[data['LTFRONT']==1,'LTFRONT']=np.nan
data.loc[data['LTDEPTH']==1,'LTDEPTH']=np.nan
data.loc[data['BLDFRONT']==1,'BLDFRONT']=np.nan
data.loc[data['BLDDEPTH']==1,'BLDDEPTH']=np.nan

In [19]:
# Finds the mean values based on the tax class
mean_LTFRONT = data.groupby(data['TAXCLASS'])['LTFRONT'].mean()
mean_LTDEPTH = data.groupby(data['TAXCLASS'])['LTDEPTH'].mean()
mean_BLDFRONT = data.groupby(data['TAXCLASS'])['BLDFRONT'].mean()
mean_BLDDEPTH = data.groupby(data['TAXCLASS'])['BLDDEPTH'].mean()

In [20]:
# Loops through dataset and inserts the mean value based on the TAXCLASS 
for index in mean_LTFRONT.index:
    data.loc[(data['LTFRONT'].isnull())&(data['TAXCLASS']==index),'LTFRONT']=mean_LTFRONT[index]
    data.loc[(data['LTDEPTH'].isnull())&(data['TAXCLASS']==index),'LTDEPTH']=mean_LTDEPTH[index]
    data.loc[(data['BLDFRONT'].isnull())&(data['TAXCLASS']==index),'BLDFRONT']=mean_BLDFRONT[index]
    data.loc[(data['BLDDEPTH'].isnull())&(data['TAXCLASS']==index),'BLDDEPTH']=mean_BLDDEPTH[index]

# Making NY Variables

In [21]:
# Sets zip codes as string value
data['ZIP'] = data['ZIP'].astype(str)
data['zip3'] = data['ZIP'].str[:3]

In [22]:
# Creates new features based on lot size, building size, and story size
data['ltsize'] = data['LTFRONT'] * data['LTDEPTH']
data['bldsize'] = data['BLDFRONT'] * data['BLDDEPTH']
data['bldvol'] = data['bldsize'] * data['STORIES']

In [23]:
# Creates nine new features as the ratio of value to the three new features we just created
data['r1'] = data['FULLVAL'] / data['ltsize']
data['r2'] = data['FULLVAL'] / data['bldsize']
data['r3'] = data['FULLVAL'] / data['bldvol']
data['r4'] = data['AVLAND'] / data['ltsize']
data['r5'] = data['AVLAND'] / data['bldsize']
data['r6'] = data['AVLAND'] / data['bldvol']
data['r7'] = data['AVTOT'] / data['ltsize']
data['r8'] = data['AVTOT'] / data['bldsize']
data['r9'] = data['AVTOT'] / data['bldvol']

In [24]:
ninevars = ['r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9']
# Finds the average of the nine new values we created across four different categorical fields
zip_average = data.groupby('ZIP')[ninevars].mean()
zip3_average = data.groupby('zip3')[ninevars].mean()
taxclass_average = data.groupby('TAXCLASS')[ninevars].mean()
b_average = data.groupby('B')[ninevars].mean()

In [25]:
# Adds new averages to the dataset
data = data.join(zip_average, on = 'ZIP', rsuffix = '_zip5')
data = data.join(zip3_average, on = 'zip3', rsuffix = '_zip3')
data = data.join(taxclass_average, on = 'TAXCLASS', rsuffix = '_taxclass')
data = data.join(b_average, on='B', rsuffix = '_boro')
rsuffix = ['_zip5', '_zip3', '_taxclass', '_boro']

In [26]:
# Finds the ratio of the nine variables to the averages of the four different categorical groupings
for var in ninevars:
    for r in rsuffix:
        data[str(var)+r] = data[var] / data[str(var)+r]

In [27]:
# subsets data to provide statistics only on the new variables that were created
subset = data[list(data.columns[-45:])]

In [28]:
# Finds summary statistics on the dataset
stats = subset.describe().transpose()

In [31]:
subset.to_csv('NYpropertydataVariablesUpdated.csv', index = False)